In [142]:
from dotenv import load_dotenv
import os

# Looking for .env file and loading it
load_dotenv() 

nyt_api = os.getenv("NYT_ID")
guardian_api =  os.getenv("GUARDIAN_ID")

# Get the id and secret from the .env
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID" )
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
genius_token = os.getenv("GENIUS_API_TOKEN")

# News Collection
Pull today's headline from the NYT API

In [1]:
! pip install requests

In [160]:
import requests

URL = f'https://api.nytimes.com/svc/topstories/v2/home.json?api-key={nyt_api}'

response = requests.get(URL)

nyt_articles = []


if response.status_code == 200:
    data = response.json()
    if data['results']:
        # Assume the first article is the main front-page article
        for article in data['results']:
            nyt_articles.append(article['title'] + ": " + article['title'])
    else:
        print("No articles found.")
else:
    print(f"Error: {response.status_code}, {response.text}")
    
print(len(nyt_articles))


32


In [163]:
import requests
from datetime import datetime


# Parameters
SECTION = 'us-news'
DATE = datetime.now().strftime('%Y-%m-%d')
URL = 'https://content.guardianapis.com/search'
NUM_ARTICLES = 8

params = {
    'section': SECTION,
    'from-date': DATE,
    'to-date': DATE,
    'order-by': 'newest',
    'page-size': NUM_ARTICLES,
    'show-fields': 'trailText',
    'api-key': guardian_api
}

guardian_articles = []

try:
    response = requests.get(URL, params=params)
    response.raise_for_status()
    data = response.json()

    if data.get('response', {}).get('status') == 'ok' and data['response']['results']:
        articles = data['response']['results']
        with open('guardian_us_headlines.txt', 'w', encoding='utf-8') as file:
            for idx, article in enumerate(articles, start=1):
                title = article.get('webTitle', 'No Title')
                abstract = article.get('fields', {}).get('trailText', 'No Abstract')
                guardian_articles.append(title + ": " + abstract)
    else:
        print("No articles found for today.")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

print(len(guardian_articles))

8


# NLP Analysis
Calculate sentiment of news articles

In [11]:
! pip install nltk

In [147]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


# NYT
stop_words = set(stopwords.words("english"))
tokenized = word_tokenize(times)
text = [word.lower() for word in tokenized if word.isalnum() and word.lower() not in stop_words]

sentiment_analyzer = SentimentIntensityAnalyzer()

news_sentiment_nyt = sentiment_analyzer.polarity_scores(' '.join(tokenized))

print("NYT Headline Sentiment:", news_sentiment_nyt['compound'])

# Guardian
tokenized = word_tokenize(guardian)
text = [word.lower() for word in tokenized if word.isalnum() and word.lower() not in stop_words]

sentiment_analyzer = SentimentIntensityAnalyzer()

news_sentiment_g = sentiment_analyzer.polarity_scores(' '.join(tokenized))

print("Guardian Headline Sentiment:", news_sentiment_g['compound'])

NYT Headline Sentiment: 0.4588
Guardian Headline Sentiment: 0.6249


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/maggiehollis/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Song Recs
Search for songs in the spotify API that have audio features we matched with the mood

In [7]:
! pip install spotipy beautifulsoup4

In [134]:

import base64
from requests import post

In [34]:
def get_token(client_id, client_secret):
    # Encode the client ID and client secret
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")
    
    # Define the URL and headers for the POST request
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {auth_base64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    # Set the data payload for the POST request
    data = {"grant_type": "client_credentials"}
    
    # Make the POST request to get the token
    result = post(url, headers=headers, data=data)
    token = result.json()
    # notice how we JSONify the Response object

    return token # Return the access token from the JSON response

In [ ]:
# Check the token
token = get_token(spotify_client_id, spotify_client_secret)
access_token = token["access_token"]

In [133]:
def get_usa_top_50(token):
    playlist_id = '6UeSakyzhiEt4NB3UAd6NQ'  # USA Top 50 Playlist ID
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}'

    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    playlist_data = response.json()

    top_50_tracks = []

    for item in playlist_data['tracks']['items']:
        track = item['track']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        top_50_tracks.append({'track': track_name, 'artist': artist_name})

    return top_50_tracks


# --- Genius API Part ---
def get_genius_url(genius_token, artist, title):
    search_url = 'https://api.genius.com/search'
    headers = {'Authorization': f'Bearer {genius_token}'}
    query = f"{artist} {title}"
    params = {'q': query}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    hits = data['response']['hits']
    if hits:
        return hits[0]['result']['url']
    return None

def get_lyrics(genius_url):
    page = requests.get(genius_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    lyrics_containers = soup.find_all('div', attrs={"data-lyrics-container": "true"})
    lyrics = "\n".join([div.get_text(strip=True) for div in lyrics_containers])
    return lyrics

usa_top_50 = get_usa_top_50(access_token)
sentiments = {}

for song in usa_top_50:
    artist = song['artist']
    title = song['track']
    print(f"\nProcessing: {artist} - {title}")
    try:
        genius_url = get_genius_url(genius_token, artist, title)
        if genius_url:
            lyrics = get_lyrics(genius_url)
            sentiment = sia.polarity_scores(lyrics)
            sentiments[f"{artist} - {title}"] = sentiment
        else:
            print("Genius URL not found.")
    except Exception as e:
        print(f"Error processing {artist} - {title}: {e}")




Processing: Kendrick Lamar - luther (with sza)

Processing: Drake - NOKIA

Processing: Lady Gaga - Die With A Smile

Processing: BigXthaPlug - All The Way (feat. Bailey Zimmerman)

Processing: Chappell Roan - Pink Pony Club

Processing: Shaboozey - A Bar Song (Tipsy)

Processing: Alex Warren - Ordinary

Processing: Teddy Swims - Lose Control

Processing: Morgan Wallen - I'm The Problem

Processing: Benson Boone - Beautiful Things

Processing: Billie Eilish - BIRDS OF A FEATHER

Processing: Morgan Wallen - Just In Case

Processing: ROSÉ - APT.

Processing: Doechii - Anxiety

Processing: Gracie Abrams - That’s So True

Processing: Kendrick Lamar - Not Like Us

Processing: Sabrina Carpenter - Espresso

Processing: Kendrick Lamar - tv off (feat. lefty gunplay)

Processing: Morgan Wallen - Love Somebody

Processing: Post Malone - I Had Some Help (Feat. Morgan Wallen)

Processing: Leon Thomas - MUTT

Processing: Lola Young - Messy

Processing: Kendrick Lamar - squabble up

Processing: Tate 

In [149]:
goal = news_sentiment_nyt['compound']
distance = 100
closest_song = "None"
closest_artist = "None"

for song, sentiment in sentiments.items():
    if abs(sentiment['compound'] - goal) < distance:
        distance = abs(sentiment['compound'] - goal)
        closest_song = song.split(" - ")[1]
        closest_artist = song.split(" - ")[0]
        
print(f"NYT: We recommend you listen to: {closest_song} by {closest_artist}")

goal = news_sentiment_g['compound']
distance = 100
closest_song = "None"
closest_artist = "None"

for song, sentiment in sentiments.items():
    if abs(sentiment['compound'] - goal) < distance:
        distance = abs(sentiment['compound'] - goal)
        closest_song = song.split(" - ")[1]
        closest_artist = song.split(" - ")[0]
        
print(f"Guardian: We recommend you listen to: {closest_song} by {closest_artist}")

NYT: We recommend you listen to: That’s So True by Gracie Abrams
Guardian: We recommend you listen to: Hometown Home by LOCASH
